# NLP Project

## Load Data

### Scraped CDP Scores
See [./download_csp_scores.py](./download_cdp_scores.py) for the CDP scores scraping code

In [18]:
import pandas as pd

esg_documents_df = pd.read_csv('./dax_esg_media_dataset/esg_documents_for_dax_companies.csv', sep='|')

cdp_scores_df = pd.read_csv('./cdp_scores.csv')

In [23]:
merged_df = pd.merge(esg_documents_df, cdp_scores_df, how='left', left_on='symbol', right_on='Ticker')

merged_df.dropna(subset=['CDP Score'], inplace=True) # drop rows without a CDP score

print(merged_df[['symbol', 'CDP Score']])
print(len(merged_df))

      symbol CDP Score
0        BEI         A
1        DTE         A
2        VNA        A-
3        MRK        A-
4        MTX         B
...      ...       ...
11543    ZAL        A-
11544    ZAL        A-
11545    ZAL        A-
11546    ZAL        A-
11547    ZAL        A-

[10198 rows x 2 columns]
10198


## Preprocessing

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    lowercase=True,
    stop_words='english',
    token_pattern=r'\b\w+\b' # Only keep word tokens
)

bow_matrix = vectorizer.fit(esg_documents_df['content'].fillna(''))
# can use the bow_matrix in the next step for building a model

## Model Fitting

In [ ]:
# TODO here we should try and build a model that correlates the preprocessed
# data to the column of our choice in the sp500_risk_ratings dataset

## Model Analysis

In [ ]:
# TODO check how well our model did here

# Data Sources
- https://www.kaggle.com/datasets/pritish509/s-and-p-500-esg-risk-ratings
- https://www.kaggle.com/datasets/equintel/dax-esg-media-dataset

# References